In [1]:
import VISSSlib
import yaml

In [42]:
settings = "/projekt1//ag_maahn/VISSS_config/hyytiala_v1.yaml"; case = "20220331-104000"
settings = "/projekt1//ag_maahn/VISSS_config/hyytiala_v1.yaml"; case = "20220402-223000"
settings = "/projekt1//ag_maahn/VISSS_config/nyaalesund_v4.yaml"; case = "20220630-090000"
settings = "/projekt1//ag_maahn/VISSS_config/mosaic_v1.yaml"; case = "20191115-174500"
version = "20220521" 


In [43]:

from importlib import reload
reload(VISSSlib.matching)
reload(VISSSlib.fixes)
reload(VISSSlib.tools)

<module 'VISSSlib.tools' from '/home/mmaahn/projectsSrv/VISSSlib/src/VISSSlib/tools.py'>

In [44]:
config = VISSSlib.tools.readSettings(settings)
fl = VISSSlib.files.FindFiles(case, config.leader, config, version)
fname1L = fl.listFiles("level1detect")[0]
fname1Match = VISSSlib.files.FilenamesFromLevel(fname1L, config).fname["level1match"]

In [45]:
rotate_default = {
    "phi": 0,
    "theta": 0,
    "Ofz": 0, 
}

rotate_err_default = {
    "phi":  5,
    "theta":  5,
    "Ofz":  50,
}

In [46]:
reload(VISSSlib.matching)
fout, matchedDat, rot, rot_err = VISSSlib.matching.matchParticles(fname1L, config, rotationOnly=True, rotate=rotate_default, rotate_err=rotate_err_default, maxDiffMs="config", nPoints=2000)


opening /projekt4/ag_maahn/data_obs_nobackup/mosaic/visss/level1detect/2019/11/15/level1detect_V20220521_mosaic_visss1_visss_leader_S1145792_20191115-174500.nc
take default values for previous rotation {'phi': 0, 'theta': 0, 'Ofz': 0}
opening ['/projekt4/ag_maahn/data_obs_nobackup/mosaic/visss/level1detect/2019/11/15/level1detect_V20220521_mosaic_visss2_visss_follower_S1143155_20191115-174500.nc']
0 of 2 slice for follower restart 2019-11-15T02:55:51.000000000 2019-11-15T10:59:34.000000000
CONTINUE, slice for follower restart 0 2019-11-15T02:55:51.000000000 2019-11-15T10:59:34.000000000 before leader time range <xarray.DataArray 'file_starttime' ()>
array('2019-11-15T17:44:59.000000000', dtype='datetime64[ns]')
1 of 2 slice for follower restart 2019-11-15T10:59:34.000000000 2019-11-15T18:04:25.000000000
using 373 of 2000
estimateCaptureIdDiff statistic: {-113391304: 373}
capture_id determined -113391304, 100.0% have the same value
using 1556 of 2000
estimateCaptureIdDiff statistic: {-1

MATCH 9 0.0039082654
ROTATE 9 phi      -0.614773
theta     0.968012
Ofz      12.354175
dtype: float64
rotation coefficients iteration 10 of 20
slicing data into 4 pieces
doMatch 1195 1057
follower_S1143155 particles have been matched twice, fixing 0
doMatch 1196 1575
leader_S1145792 particles have been matched twice, fixing 0
follower_S1143155 particles have been matched twice, fixing 0
doMatch 1195 918
leader_S1145792 particles have been matched twice, fixing 0
follower_S1143155 particles have been matched twice, fixing 0
doMatch 1196 1815
leader_S1145792 particles have been matched twice, fixing 0
leader_S1145792 particles have been matched twice, fixing 1
follower_S1143155 particles have been matched twice, fixing 0
follower_S1143155 particles have been matched twice, fixing 1
MATCH 10 0.0037705165
ROTATE 10 phi      -0.576556
theta     0.980522
Ofz      11.020808
dtype: float64
rotation coefficients iteration 11 of 20
slicing data into 4 pieces
doMatch 1195 1057
follower_S1143155 p

leader_S1145792 particles have been matched twice, fixing 0
leader_S1145792 particles have been matched twice, fixing 1
follower_S1143155 particles have been matched twice, fixing 0
follower_S1143155 particles have been matched twice, fixing 1
MATCH 19 0.003950293
ROTATE 19 phi      0.005503
theta    1.041985
Ofz     -1.478743
dtype: float64


In [47]:
rot, rot_err

(phi      0.005503
 theta    1.041985
 Ofz     -1.478743
 dtype: float64,
 phi      0.016855
 theta    0.016414
 Ofz      0.339828
 dtype: float64)

In [48]:
fout, matchedDat, rot2, rot_err2 = VISSSlib.matching.matchParticles(fname1L, config, rotationOnly=True, rotate=rot, rotate_err=rot_err, maxDiffMs=3.6, nPoints=500)
rot2, rot_err2

opening /projekt4/ag_maahn/data_obs_nobackup/mosaic/visss/level1detect/2019/11/15/level1detect_V20220521_mosaic_visss1_visss_leader_S1145792_20191115-174500.nc
take default values for previous rotation phi      0.005503
theta    1.041985
Ofz     -1.478743
dtype: float64
opening ['/projekt4/ag_maahn/data_obs_nobackup/mosaic/visss/level1detect/2019/11/15/level1detect_V20220521_mosaic_visss2_visss_follower_S1143155_20191115-174500.nc']
0 of 2 slice for follower restart 2019-11-15T02:55:51.000000000 2019-11-15T10:59:34.000000000
CONTINUE, slice for follower restart 0 2019-11-15T02:55:51.000000000 2019-11-15T10:59:34.000000000 before leader time range <xarray.DataArray 'file_starttime' ()>
array('2019-11-15T17:44:59.000000000', dtype='datetime64[ns]')
1 of 2 slice for follower restart 2019-11-15T10:59:34.000000000 2019-11-15T18:04:25.000000000
using 99 of 500
estimateCaptureIdDiff statistic: {-113391304: 99}
capture_id determined -113391304, 100.0% have the same value
using 387 of 500
estim

leader_S1145792 particles have been matched twice, fixing 0
leader_S1145792 particles have been matched twice, fixing 1
follower_S1143155 particles have been matched twice, fixing 0
follower_S1143155 particles have been matched twice, fixing 1
doMatch 1196 1815
leader_S1145792 particles have been matched twice, fixing 0
leader_S1145792 particles have been matched twice, fixing 1
follower_S1143155 particles have been matched twice, fixing 0
follower_S1143155 particles have been matched twice, fixing 1
MATCH 7 0.0049551614
ROTATE 7 phi       0.589344
theta     0.618858
Ofz     -19.630888
dtype: float64
rotation coefficients iteration 8 of 20
slicing data into 4 pieces
doMatch 1195 1057
leader_S1145792 particles have been matched twice, fixing 0
follower_S1143155 particles have been matched twice, fixing 0
doMatch 1196 1575
leader_S1145792 particles have been matched twice, fixing 0
follower_S1143155 particles have been matched twice, fixing 0
doMatch 1195 918
leader_S1145792 particles ha

MATCH 15 0.0055206525
ROTATE 15 phi       0.867353
theta    -0.293178
Ofz     -39.541914
dtype: float64
rotation coefficients iteration 16 of 20
slicing data into 4 pieces
doMatch 1195 1057
leader_S1145792 particles have been matched twice, fixing 0
follower_S1143155 particles have been matched twice, fixing 0
doMatch 1196 1575
leader_S1145792 particles have been matched twice, fixing 0
follower_S1143155 particles have been matched twice, fixing 0
doMatch 1195 918
leader_S1145792 particles have been matched twice, fixing 0
leader_S1145792 particles have been matched twice, fixing 1
follower_S1143155 particles have been matched twice, fixing 0
follower_S1143155 particles have been matched twice, fixing 1
follower_S1143155 particles have been matched twice, fixing 2
doMatch 1196 1815
leader_S1145792 particles have been matched twice, fixing 0
leader_S1145792 particles have been matched twice, fixing 1
leader_S1145792 particles have been matched twice, fixing 2
leader_S1145792 particles h

(phi       0.974067
 theta    -0.913437
 Ofz     -52.701971
 dtype: float64,
 phi      0.014350
 theta    0.026300
 Ofz      0.492528
 dtype: float64)

In [52]:
fout, matchedDat, rot3, rot_err3 = VISSSlib.matching.matchParticles(fname1L, config, rotationOnly=True, rotate=rot2, rotate_err=rot_err2, maxDiffMs=3.6, nPoints=500)


opening /projekt4/ag_maahn/data_obs_nobackup/mosaic/visss/level1detect/2019/11/15/level1detect_V20220521_mosaic_visss1_visss_leader_S1145792_20191115-174500.nc
take default values for previous rotation phi       0.974067
theta    -0.913437
Ofz     -52.701971
dtype: float64
opening ['/projekt4/ag_maahn/data_obs_nobackup/mosaic/visss/level1detect/2019/11/15/level1detect_V20220521_mosaic_visss2_visss_follower_S1143155_20191115-174500.nc']
0 of 2 slice for follower restart 2019-11-15T02:55:51.000000000 2019-11-15T10:59:34.000000000
CONTINUE, slice for follower restart 0 2019-11-15T02:55:51.000000000 2019-11-15T10:59:34.000000000 before leader time range <xarray.DataArray 'file_starttime' ()>
array('2019-11-15T17:44:59.000000000', dtype='datetime64[ns]')
1 of 2 slice for follower restart 2019-11-15T10:59:34.000000000 2019-11-15T18:04:25.000000000
using 99 of 500
estimateCaptureIdDiff statistic: {-113391304: 99}
capture_id determined -113391304, 100.0% have the same value
using 387 of 500
es

In [53]:
print(yaml.dump({"rotate":rot3.round(6).to_dict(), "rotate_err": rot_err3.round(6).to_dict()}))

rotate:
  Ofz: -59.87029
  phi: 1.012026
  theta: -1.292008
rotate_err:
  Ofz: 0.238301
  phi: 0.01551
  theta: 0.016509



In [54]:
fout, matchedDat, rot4, rot_err4 = VISSSlib.matching.matchParticles(fname1L, config, rotationOnly=True, rotate=rot3, rotate_err=rot_err3, maxDiffMs=3.6, nPoints=500)


opening /projekt4/ag_maahn/data_obs_nobackup/mosaic/visss/level1detect/2019/11/15/level1detect_V20220521_mosaic_visss1_visss_leader_S1145792_20191115-174500.nc
take default values for previous rotation phi       1.012026
theta    -1.292008
Ofz     -59.870290
dtype: float64
opening ['/projekt4/ag_maahn/data_obs_nobackup/mosaic/visss/level1detect/2019/11/15/level1detect_V20220521_mosaic_visss2_visss_follower_S1143155_20191115-174500.nc']
0 of 2 slice for follower restart 2019-11-15T02:55:51.000000000 2019-11-15T10:59:34.000000000
CONTINUE, slice for follower restart 0 2019-11-15T02:55:51.000000000 2019-11-15T10:59:34.000000000 before leader time range <xarray.DataArray 'file_starttime' ()>
array('2019-11-15T17:44:59.000000000', dtype='datetime64[ns]')
1 of 2 slice for follower restart 2019-11-15T10:59:34.000000000 2019-11-15T18:04:25.000000000
using 99 of 500
estimateCaptureIdDiff statistic: {-113391304: 99}
capture_id determined -113391304, 100.0% have the same value
using 387 of 500
es

In [55]:
print(yaml.dump({"rotate":rot4.round(6).to_dict(), "rotate_err": rot_err4.round(6).to_dict()}))

rotate:
  Ofz: -60.109224
  phi: 1.019082
  theta: -1.30352
rotate_err:
  Ofz: 0.235507
  phi: 0.015808
  theta: 0.01641

